In [1]:
import wget
import numpy as np
import pandas as pd
from tqdm import tqdm

from abc import *
from pathlib import Path
import os
import tempfile
import shutil
import pickle
import json
import pprint as pp

import pandas as pd
import time 
from datetime import datetime, date

from abc import *
import random
import torch
import torch.utils.data as data_utils

### Model Building
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import math
import torch.nn.functional as F



In [2]:
# Static Class that will hold our parameter values as attributes, thus making it easy to call

class Parameters:
    def __init__(self):
        self.random_seed = 10
        
        #Data Split
        self.train_size = 0.8
        self.test_size = 1- self.train_size
        self.save_folder = '../data/bert_4_rec_data'

        #Sampling
        self.train_negative_sampler_code = 'random'
        self.train_negative_sample_size = 0
        self.train_negative_sampling_seed = 0
        self.test_negative_sampler_code = 'random'
        self.test_negative_sample_size = 100
        self.test_negative_sampling_seed = 98765

        #BERT Model
        self.bert_max_len = 100
        self.bert_mask_prob = 0.15
        self.batch_size = 128

        self.bert_dropout = 0.1
        self.bert_hidden_units = 256
        self.bert_mask_prob = 0.15
        self.bert_max_len = 100
        self.bert_num_blocks = 2
        self.bert_num_heads = 4

        # Untunable Parameters:
        self.num_items =None

        # Parameters for trainer:
        self.num_gpu = 1
        self.device = 'cuda'
        self.device_idx = '0'
        self.optimizer = 'Adam'
        self.lr = 0.001
        self.enable_lr_schedule = True
        self.decay_step = 25
        self.gamma = 1.0
        self.num_epochs = 100
        self.metric_ks = [1, 5, 10, 20, 50, 100]
        self.best_metric = 'NDCG@10'
        self.log_period_as_iter = 10000 # Define how often to log data 
        
        #Parameters for Optimizer:
        self.weight_decay = 0

        # Saving Model:
        self.experiment_dir = "model"
        self.experiment_description = "BT4222"

args = Parameters()

In [3]:
tqdm.pandas()

C:\Users\ngtze\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
try:
    dataset = pd.read_pickle('../data/bert_4_rec_data/dataset.pkl')

except:
    data = pd.read_csv('../data/processed_data/full_data.csv')
    data = data.drop(columns = "Unnamed: 0")

    ## Retriving the relevant features
    ratings = pd.DataFrame()

    ratings['userId'] = data['userId']
    ratings['movieId'] = data['movieId']
    ratings['rating'] = data['rating']
    ratings['timestamp'] = data['timestamp']

    # Since we are considering good movies we will filter out those rating below 3.5 (Determined by the histogram of ratings)
    ratings = ratings[ratings['rating'] >= 3.5]

    # Probability Density  Distribution of user and movie

    print('Densifying index')
    umap = {u: i for i, u in enumerate(set(ratings['userId']))}
    smap = {s: i for i, s in enumerate(set(ratings['movieId']))}
    ratings['userId'] = ratings['userId'].map(umap)
    ratings['movieId'] = ratings['movieId'].map(smap)

    # Since we are considering the changing user preference overtime, we have to model the data as a timeseries and split train/test by specifying a specific timepoint to split the data (leave-one-Out Sampling)
    user_count = len(umap)
    print('Splitting')
    user_group = ratings.groupby('userId')
    user2items = user_group.progress_apply(lambda d: list(d.sort_values(by='timestamp')['movieId']))
    train, val, test = {}, {}, {}
    for user in range(user_count):
        items = user2items[user]
        train[user], val[user], test[user] = items[:-2], items[-2:-1], items[-1:]

    dataset = {'train': train,
            'val': val,
            'test': test,
            'umap': umap,
            'smap': smap}

    with open('../data/bert_4_rec_data/dataset.pkl', "wb") as e:
        pickle.dump(dataset, e)

In [5]:
### Implement a Negative Sampling Class
from tqdm import trange

from collections import Counter

class PopularNegativeSampler(metaclass=ABCMeta):

    def __init__(self, train, val, test, user_count, item_count, sample_size, seed, save_folder):
        self.train = train
        self.val = val
        self.test = test
        self.user_count = user_count
        self.item_count = item_count
        self.sample_size = sample_size
        self.seed = seed
        self.save_folder = save_folder
    
    def get_negative_samples(self):
        savefile_path = self._get_save_path()
        if savefile_path.is_file():
            print('Negatives samples exist. Loading.')
            negative_samples = pickle.load(savefile_path.open('rb'))
            return negative_samples
        print("Negative samples don't exist. Generating.")
        negative_samples = self.generate_negative_samples()
        with savefile_path.open('wb') as f:
            pickle.dump(negative_samples, f)
        return negative_samples

    def _get_save_path(self):
        folder = Path(self.save_folder)
        filename = 'Popular-sample_size{}-seed{}.pkl'.format(self.sample_size, self.seed)
        return folder.joinpath(filename)

    def generate_negative_samples(self):
        popular_items = self.items_by_popularity()

        negative_samples = {}
        print('Sampling negative items')
        for user in trange(self.user_count):
            try:
                seen = set(self.train[user])
                seen.update(self.val[user])
                seen.update(self.test[user])

                samples = []
                for item in popular_items:
                    if len(samples) == self.sample_size:
                        break
                    if item in seen:
                        continue
                    samples.append(item)

                negative_samples[user] = samples
            except:
                continue

        return negative_samples

    def items_by_popularity(self):
        popularity = Counter()

        for user in self.train.keys():
            popularity.update(self.train[user])

        for user in self.val.keys():
            popularity.update(self.val[user])

        for user in self.test.keys():
            popularity.update(self.test[user])
            
        popular_items = sorted(popularity, key=popularity.get, reverse=True)
        return popular_items

In [6]:
### Implement a Dataloader class that will load data into model
from abc import *
import random

class model_Dataloader(metaclass=ABCMeta):
    def __init__(self, args, dataset):
        self.args = args
        seed = args.random_seed
        self.rng = random.Random(seed)
        self.train = dataset['train']
        self.val = dataset['val']
        self.test = dataset['test']
        self.umap = dataset['umap']
        self.smap = dataset['smap']
        self.user_count = len(self.umap)
        self.item_count = len(self.smap)
        self.max_len = args.bert_max_len
        self.mask_prob = args.bert_mask_prob
        self.CLOZE_MASK_TOKEN = self.item_count + 1
        self.save_folder = args.save_folder
        args.num_items = len(self.smap)

        code = args.train_negative_sampler_code
        train_negative_sampler = PopularNegativeSampler(self.train, self.val, self.test,
                                                          self.user_count, self.item_count,
                                                          args.train_negative_sample_size,
                                                          args.train_negative_sampling_seed,
                                                          self.save_folder)
        code = args.test_negative_sampler_code
        test_negative_sampler = PopularNegativeSampler(self.train, self.val, self.test,
                                                         self.user_count, self.item_count,
                                                         args.test_negative_sample_size,
                                                         args.test_negative_sampling_seed,
                                                         self.save_folder)

        self.train_negative_samples = train_negative_sampler.get_negative_samples()
        self.test_negative_samples = test_negative_sampler.get_negative_samples()

    def code(cls):
        return 'bert'

    def get_pytorch_dataloaders(self):
        train_loader = self._get_train_loader()
        val_loader = self._get_val_loader()
        test_loader = self._get_test_loader()
        return train_loader, val_loader, test_loader

    def _get_train_loader(self):
        dataset = self._get_train_dataset()
        dataloader = data_utils.DataLoader(dataset, batch_size=self.args.batch_size,
                                           shuffle=True, pin_memory=True)
        return dataloader

    def _get_train_dataset(self):
        dataset = BertTrainDataset(self.train, self.max_len, self.mask_prob, self.CLOZE_MASK_TOKEN, self.item_count, self.rng)
        return dataset

    def _get_val_loader(self):
        return self._get_eval_loader(mode='val')

    def _get_test_loader(self):
        return self._get_eval_loader(mode='test')

    def _get_eval_loader(self, mode):
        batch_size = self.args.batch_size
        dataset = self._get_eval_dataset(mode)
        dataloader = data_utils.DataLoader(dataset, batch_size=batch_size,
                                           shuffle=False, pin_memory=True)
        return dataloader

    def _get_eval_dataset(self, mode):
        answers = self.val if mode == 'val' else self.test
        dataset = BertEvalDataset(self.train, answers, self.max_len, self.CLOZE_MASK_TOKEN, self.test_negative_samples)
        return dataset


class BertTrainDataset(data_utils.Dataset):
    def __init__(self, u2seq, max_len, mask_prob, mask_token, num_items, rng):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys())
        self.max_len = max_len
        self.mask_prob = mask_prob
        self.mask_token = mask_token
        self.num_items = num_items
        self.rng = rng

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self._getseq(user)

        tokens = []
        labels = []
        for s in seq:
            prob = self.rng.random()
            if prob < self.mask_prob:
                prob /= self.mask_prob

                if prob < 0.8:
                    tokens.append(self.mask_token)
                elif prob < 0.9:
                    tokens.append(self.rng.randint(1, self.num_items))
                else:
                    tokens.append(s)

                labels.append(s)
            else:
                tokens.append(s)
                labels.append(0)

        tokens = tokens[-self.max_len:]
        labels = labels[-self.max_len:]

        mask_len = self.max_len - len(tokens)

        tokens = [0] * mask_len + tokens
        labels = [0] * mask_len + labels

        return torch.LongTensor(tokens), torch.LongTensor(labels)

    def _getseq(self, user):
        return self.u2seq[user]


class BertEvalDataset(data_utils.Dataset):
    def __init__(self, u2seq, u2answer, max_len, mask_token, negative_samples):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys())
        self.u2answer = u2answer
        self.max_len = max_len
        self.mask_token = mask_token
        self.negative_samples = negative_samples

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self.u2seq[user]
        answer = self.u2answer[user]
        negs = self.negative_samples[user]

        candidates = answer + negs
        labels = [1] * len(answer) + [0] * len(negs)

        seq = seq + [self.mask_token]
        seq = seq[-self.max_len:]
        padding_len = self.max_len - len(seq)
        seq = [0] * padding_len + seq

        return torch.LongTensor(seq), torch.LongTensor(candidates), torch.LongTensor(labels)


In [7]:
dataloader = model_Dataloader(args, dataset)

Negatives samples exist. Loading.
Negatives samples exist. Loading.


In [8]:
train, val, test = dataloader.get_pytorch_dataloaders()


## BERT Model Implementation as adapted from: https://github.com/codertimo/BERT-pytorch/tree/master/bert_pytorch/model

### Embedding

Embedding Layer BERT consists of 3 Embedding Layers <br>
1. Token Embedding : normal embedding matrix <br>
2. Positional Embedding : adding positional information <br>
3. Segment Embedding : adding sentence segment info, (sent_A:1, sent_B:2)<br>

Embedding size is 512 as BERT was designed to process input sequences of up to length 512 <br>

Implemented using pytorch embedding class: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

More on Embedding can be found : https://medium.com/@_init_/why-bert-has-3-embedding-layers-and-their-implementation-details-9c261108e28a


Token Embedding -> Word Piece Tokenization -> torch normal Embedding

In [9]:
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)

Positional Embedding

In [10]:
class PositionalEmbedding(nn.Module):

    def __init__(self, max_len, d_model):
        super().__init__()

        # Compute the positional encodings once in log space.
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size = x.size(0)
        return self.pe.weight.unsqueeze(0).repeat(batch_size, 1, 1)

Segment Embedding

In [11]:
class SegmentEmbedding(nn.Embedding):
    def __init__(self, embed_size=512):
        super().__init__(3, embed_size, padding_idx=0)

BERT Embedding

In [12]:
class BERTEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size, max_len, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """
        super().__init__()
        #Token Embedding
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(max_len=max_len, d_model=embed_size)
        # self.segment = SegmentEmbedding(embed_size=self.token.embedding_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    def forward(self, sequence):
        x = self.token(sequence) + self.position(sequence)  # + self.segment(segment_label)
        return self.dropout(x)

### Transformer

The activation function used in BERT is GELU: <br>
More about it can be found : https://paperswithcode.com/method/gelu <br>

In [13]:
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

In [14]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = GELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))

Based on the paper : https://arxiv.org/abs/1607.06450 <br>

Layer Normalization is often used in transformer achitecture. <br>

The idea of normalization is to improve the model's ability to perform on unseen data (Generalization) <br>
as well as reduce the training time of the model

The rationale for implementing Layer Normalization over Batch Normalization is that the <br>
effectiveness of batch normalization is dependent on batch size.

Layer Normalization is implemented with torch Layer Norm Class : https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html

In [15]:
class LayerNorm(nn.Module):

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [16]:
class SublayerConnection(nn.Module):
    
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [17]:
class TransformerBlock(nn.Module):
    """
    Bidirectional Encoder = Transformer (self-attention)
    Transformer = MultiHead_Attention + Feed_Forward with sublayer connection
    """

    def __init__(self, hidden, attn_heads, feed_forward_hidden, dropout):
        """
        :param hidden: hidden size of transformer
        :param attn_heads: head sizes of multi-head attention
        :param feed_forward_hidden: feed_forward_hidden, usually 4*hidden_size
        :param dropout: dropout rate
        """

        super().__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model=hidden, d_ff=feed_forward_hidden, dropout=dropout)
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return self.dropout(x)

### Attention Layer

In [18]:
class Attention(nn.Module):
    """
    Compute 'Scaled Dot Product Attention
    """

    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) \
                 / math.sqrt(query.size(-1))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn

In [19]:
class MultiHeadedAttention(nn.Module):
    """
    Take in model size and number of heads.
    """

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h

        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linear_layers, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x)

In [20]:
class BERT(nn.Module):
    def __init__(self, args):
        super().__init__()

        random.seed(args.random_seed)
        torch.manual_seed(args.random_seed)
        torch.cuda.manual_seed_all(args.random_seed)
        np.random.seed(args.random_seed)
        cudnn.deterministic = True
        cudnn.benchmark = False

        max_len = args.bert_max_len
        num_items = args.num_items
        n_layers = args.bert_num_blocks
        heads = args.bert_num_heads
        vocab_size = num_items + 2
        hidden = args.bert_hidden_units
        self.hidden = hidden
        dropout = args.bert_dropout

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=self.hidden, max_len=max_len, dropout=dropout)

        # multi-layers transformer blocks, deep network
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(hidden, heads, hidden * 4, dropout) for _ in range(n_layers)])

    def forward(self, x):
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x)

        # running over multiple transformer blocks
        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)

        return x

    def init_weights(self):
        pass

### Putting it all together --BERT Model--

In [21]:
class BERTModel(nn.Module, metaclass=ABCMeta):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.bert = BERT(args)
        self.out = nn.Linear(self.bert.hidden, args.num_items + 1)

    def forward(self, x):
        x = self.bert(x)
        return self.out(x)

In [22]:
model = BERTModel(args)

## Build a trainer class that will facilitate the training of the model

In [23]:
STATE_DICT_KEY = 'model_state_dict'
OPTIMIZER_STATE_DICT_KEY = 'optimizer_state_dict'

## Define the Scoring metric that will be used

Recall :

Nomarlized Discounted Cumulative Gain :


#### Recall

In [24]:
def recall(scores, labels, k):
    scores = scores
    labels = labels
    rank = (-scores).argsort(dim=1)
    cut = rank[:, :k]
    hit = labels.gather(1, cut)
    return (hit.sum(1).float() / torch.min(torch.Tensor([k]).to(hit.device), labels.sum(1).float())).mean().cpu().item()

#### Nomarlized Discounted Cumulative Gain

In [25]:
def ndcg(scores, labels, k):
    scores = scores.cpu()
    labels = labels.cpu()
    rank = (-scores).argsort(dim=1)
    cut = rank[:, :k]
    hits = labels.gather(1, cut)
    position = torch.arange(2, 2+k)
    weights = 1 / torch.log2(position.float())
    dcg = (hits.float() * weights).sum(1)
    idcg = torch.Tensor([weights[:min(int(n), k)].sum() for n in labels.sum(1)])
    ndcg = dcg / idcg
    return ndcg.mean()

### Scoring Metric in terms of predicting the top k movies

In [26]:
def recalls_and_ndcgs_for_ks(scores, labels, ks):
    metrics = {}

    scores = scores
    labels = labels
    answer_count = labels.sum(1)

    labels_float = labels.float()
    rank = (-scores).argsort(dim=1)
    cut = rank
    for k in sorted(ks, reverse=True):
       cut = cut[:, :k]
       hits = labels_float.gather(1, cut)
       metrics['Recall@%d' % k] = \
           (hits.sum(1) / torch.min(torch.Tensor([k]).to(labels.device), labels.sum(1).float())).mean().cpu().item()

       position = torch.arange(2, 2+k)
       weights = 1 / torch.log2(position.float())
       dcg = (hits * weights.to(hits.device)).sum(1)
       idcg = torch.Tensor([weights[:min(int(n), k)].sum() for n in answer_count]).to(dcg.device)
       ndcg = (dcg / idcg).mean()
       metrics['NDCG@%d' % k] = ndcg.cpu().item()

    return metrics

## Define a AverageMeter class to track the performance of the model

In [27]:
## Define a AverageMeter class to track the performance of the model

class AverageMeterSet(object):
    def __init__(self, meters=None):
        self.meters = meters if meters else {}

    def __getitem__(self, key):
        if key not in self.meters:
            meter = AverageMeter()
            meter.update(0)
            return meter
        return self.meters[key]

    def update(self, name, value, n=1):
        if name not in self.meters:
            self.meters[name] = AverageMeter()
        self.meters[name].update(value, n)

    def reset(self):
        for meter in self.meters.values():
            meter.reset()

    def values(self, format_string='{}'):
        return {format_string.format(name): meter.val for name, meter in self.meters.items()}

    def averages(self, format_string='{}'):
        return {format_string.format(name): meter.avg for name, meter in self.meters.items()}

    def sums(self, format_string='{}'):
        return {format_string.format(name): meter.sum for name, meter in self.meters.items()}

    def counts(self, format_string='{}'):
        return {format_string.format(name): meter.count for name, meter in self.meters.items()}


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count

    def __format__(self, format):
        return "{self.val:{format}} ({self.avg:{format}})".format(self=self, format=format)

## Define Logger Class that stores best states of the model

In [28]:
def save_state_dict(state_dict, path, filename):
    torch.save(state_dict, os.path.join(path, filename))

### Administrative codes to facilitate the saving of model and results to file directories

In [29]:
def create_experiment_export_folder(args):
    experiment_dir, experiment_description = args.experiment_dir, args.experiment_description
    if not os.path.exists(experiment_dir):
        os.mkdir(experiment_dir)
    experiment_path = get_name_of_experiment_path(experiment_dir, experiment_description)
    os.mkdir(experiment_path)
    print('Folder created: ' + os.path.abspath(experiment_path))
    return experiment_path


def get_name_of_experiment_path(experiment_dir, experiment_description):
    experiment_path = os.path.join(experiment_dir, (experiment_description + "_" + str(date.today())))
    idx = _get_experiment_index(experiment_path)
    experiment_path = experiment_path + "_" + str(idx)
    return experiment_path


def _get_experiment_index(experiment_path):
    idx = 0
    while os.path.exists(experiment_path + "_" + str(idx)):
        idx += 1
    return idx

def save_test_result(export_root, result):
    filepath = Path(export_root).joinpath('test_result.txt')
    with filepath.open('w') as f:
        json.dump(result, f, indent=2)


def export_experiments_config_as_json(args, experiment_path):
    with open(os.path.join(experiment_path, 'config.json'), 'w') as outfile:
        json.dump(vars(args), outfile, indent=2)


def load_pretrained_weights(model, path):
    chk_dict = torch.load(os.path.abspath(path))
    model_state_dict = chk_dict[STATE_DICT_KEY] if STATE_DICT_KEY in chk_dict else chk_dict['state_dict']
    model.load_state_dict(model_state_dict)

In [30]:
class ModelLogger(metaclass=ABCMeta):
    def __init__(self, checkpoint_path, metric_key='mean_iou', filename='best_acc_model.pth'):
        self.checkpoint_path = checkpoint_path
        if not os.path.exists(self.checkpoint_path):
            os.mkdir(self.checkpoint_path)

        self.best_metric = 0.
        self.metric_key = metric_key
        self.filename = filename

    def log(self, *args, **kwargs):
        current_metric = kwargs[self.metric_key]
        if self.best_metric < current_metric:
            print("Update Best {} Model at {}".format(self.metric_key, kwargs['epoch']))
            self.best_metric = current_metric
            save_state_dict(kwargs['state_dict'], self.checkpoint_path, self.filename)
    
    def complete(self, *args, **kwargs):
        save_state_dict(kwargs['state_dict'], self.checkpoint_path, self.filename + '.final')

In [31]:
class LoggerService(object):
    def __init__(self, train_loggers=None, val_loggers=None):
        self.train_loggers = train_loggers if train_loggers else []
        self.val_loggers = val_loggers if val_loggers else []

    def complete(self, log_data):
        for logger in self.train_loggers:
            logger.complete(**log_data)
        for logger in self.val_loggers:
            logger.complete(**log_data)

    def log_train(self, log_data):
        for logger in self.train_loggers:
            logger.log(**log_data)

    def log_val(self, log_data):
        for logger in self.val_loggers:
            logger.log(**log_data)

In [32]:
class MetricGraphPrinter(metaclass=ABCMeta):
    def __init__(self, writer, key='train_loss', graph_name='Train Loss', group_name='metric'):
        self.key = key
        self.graph_label = graph_name
        self.group_name = group_name
        self.writer = writer

    def log(self, *args, **kwargs):
        if self.key in kwargs:
            self.writer.add_scalar(self.group_name + '/' + self.graph_label, kwargs[self.key], kwargs['accum_iter'])
        else:
            self.writer.add_scalar(self.group_name + '/' + self.graph_label, 0, kwargs['accum_iter'])

    def complete(self, *args, **kwargs):
        self.writer.close()

## Trainer Class Implementation

In [33]:

class BERTTrainer(metaclass=ABCMeta):
    def __init__(self, args, model, train_loader, val_loader, test_loader, export_root):
        self.args = args
        self.device = args.device
        self.model = model.to(self.device)
        self.is_parallel = args.num_gpu > 1
        if self.is_parallel:
            self.model = nn.DataParallel(self.model)

        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.optimizer = self._create_optimizer()
        if args.enable_lr_schedule:
            self.lr_scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=args.decay_step, gamma=args.gamma)

        self.num_epochs = args.num_epochs
        self.metric_ks = args.metric_ks
        self.best_metric = args.best_metric

        self.export_root = export_root
        self.writer, self.train_loggers, self.val_loggers = self._create_loggers()

        self.logger_service = LoggerService(self.train_loggers, self.val_loggers)
        self.log_period_as_iter = args.log_period_as_iter
        self.ce = nn.CrossEntropyLoss(ignore_index=0)

    def train(self):
        accum_iter = 0
        self.validate(0, accum_iter)
        for epoch in range(self.num_epochs):
            accum_iter = self.train_one_epoch(epoch, accum_iter)
            self.validate(epoch, accum_iter)
        self.logger_service.complete({
            'state_dict': (self._create_state_dict()),
        })
        self.writer.close()
    
    def train_one_epoch(self, epoch, accum_iter):
        self.model.train()
        if self.args.enable_lr_schedule:
            self.lr_scheduler.step()

        average_meter_set = AverageMeterSet()
        tqdm_dataloader = tqdm(self.train_loader)

        for batch_idx, batch in enumerate(tqdm_dataloader):
            batch_size = batch[0].size(0)
            batch = [x.to(self.device) for x in batch]

            self.optimizer.zero_grad()
            loss = self.calculate_loss(batch)
            loss.backward()

            self.optimizer.step()

            average_meter_set.update('loss', loss.item())
            tqdm_dataloader.set_description(
                'Epoch {}, loss {:.3f} '.format(epoch+1, average_meter_set['loss'].avg))

            accum_iter += batch_size

            if self._needs_to_log(accum_iter):
                tqdm_dataloader.set_description('Logging to Tensorboard')
                log_data = {
                    'state_dict': (self._create_state_dict()),
                    'epoch': epoch+1,
                    'accum_iter': accum_iter,
                }
                log_data.update(average_meter_set.averages())
                self.logger_service.log_train(log_data)

        return accum_iter

    def validate(self, epoch, accum_iter):
        self.model.eval()

        average_meter_set = AverageMeterSet()

        with torch.no_grad():
            tqdm_dataloader = tqdm(self.val_loader)
            for batch_idx, batch in enumerate(tqdm_dataloader):
                batch = [x.to(self.device) for x in batch]

                metrics = self.calculate_metrics(batch)

                for k, v in metrics.items():
                    average_meter_set.update(k, v)
                description_metrics = ['NDCG@%d' % k for k in self.metric_ks[:3]] +\
                                      ['Recall@%d' % k for k in self.metric_ks[:3]]
                description = 'Val: ' + ', '.join(s + ' {:.3f}' for s in description_metrics)
                description = description.replace('NDCG', 'N').replace('Recall', 'R')
                description = description.format(*(average_meter_set[k].avg for k in description_metrics))
                tqdm_dataloader.set_description(description)

            log_data = {
                'state_dict': (self._create_state_dict()),
                'epoch': epoch+1,
                'accum_iter': accum_iter,
            }
            log_data.update(average_meter_set.averages())
            self.logger_service.log_val(log_data)
    
    def test(self):
        print('Test best model with test set!')

        best_model = torch.load(os.path.join(self.export_root, 'models', 'best_acc_model.pth')).get('model_state_dict')
        self.model.load_state_dict(best_model)
        self.model.eval()

        average_meter_set = AverageMeterSet()

        with torch.no_grad():
            tqdm_dataloader = tqdm(self.test_loader)
            for batch_idx, batch in enumerate(tqdm_dataloader):
                batch = [x.to(self.device) for x in batch]

                metrics = self.calculate_metrics(batch)

                for k, v in metrics.items():
                    average_meter_set.update(k, v)
                description_metrics = ['NDCG@%d' % k for k in self.metric_ks[:3]] +\
                                      ['Recall@%d' % k for k in self.metric_ks[:3]]
                description = 'Val: ' + ', '.join(s + ' {:.3f}' for s in description_metrics)
                description = description.replace('NDCG', 'N').replace('Recall', 'R')
                description = description.format(*(average_meter_set[k].avg for k in description_metrics))
                tqdm_dataloader.set_description(description)

            average_metrics = average_meter_set.averages()
            with open(os.path.join(self.export_root, 'logs', 'test_metrics.json'), 'w') as f:
                json.dump(average_metrics, f, indent=4)
            print(average_metrics)

    def _create_optimizer(self):
        return optim.Adam(self.model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

    def calculate_loss(self, batch):
        seqs, labels = batch
        logits = self.model(seqs)  # B x T x V

        logits = logits.view(-1, logits.size(-1))  # (B*T) x V
        labels = labels.view(-1)  # B*T
        loss = self.ce(logits, labels)
        return loss

    def calculate_metrics(self, batch):
        seqs, candidates, labels = batch
        scores = self.model(seqs)  # B x T x V
        scores = scores[:, -1, :]  # B x V
        scores = scores.gather(1, candidates)  # B x C

        metrics = recalls_and_ndcgs_for_ks(scores, labels, self.metric_ks)
        return metrics
    
    def _create_loggers(self):
        root = Path(self.export_root)
        writer = SummaryWriter(root.joinpath('logs'))
        model_checkpoint = root.joinpath('models')

        train_loggers = [
            MetricGraphPrinter(writer, key='epoch', graph_name='Epoch', group_name='Train'),
            MetricGraphPrinter(writer, key='loss', graph_name='Loss', group_name='Train'),
        ]

        val_loggers = []
        for k in self.metric_ks:
            val_loggers.append(
                MetricGraphPrinter(writer, key='NDCG@%d' % k, graph_name='NDCG@%d' % k, group_name='Validation'))
            val_loggers.append(
                MetricGraphPrinter(writer, key='Recall@%d' % k, graph_name='Recall@%d' % k, group_name='Validation'))
        val_loggers.append(ModelLogger(model_checkpoint, metric_key=self.best_metric))
        return writer, train_loggers, val_loggers

    def _create_state_dict(self):
        return {
            STATE_DICT_KEY: self.model.module.state_dict() if self.is_parallel else self.model.state_dict(),
            OPTIMIZER_STATE_DICT_KEY: self.optimizer.state_dict(),
        }

    def _needs_to_log(self, accum_iter):
        return accum_iter % self.log_period_as_iter < self.args.batch_size and accum_iter != 0

### Intializing the Classes for Model Training

In [34]:
def setup_train(args):

    export_root = create_experiment_export_folder(args)
    export_experiments_config_as_json(args, export_root)

    pp.pprint({k: v for k, v in vars(args).items() if v is not None}, width=1)
    return export_root

In [35]:
export_root = setup_train(args)
dataloader = model_Dataloader(args, dataset)
train, val, test = dataloader.get_pytorch_dataloaders()
model = BERTModel(args)
trainer = BERTTrainer(args, model, train, val, test, export_root)

Folder created: c:\Users\ngtze\Desktop\Bert4Rec\Bert4Rec\model\BT4222_2021-04-10_0
{'batch_size': 128,
 'bert_dropout': 0.1,
 'bert_hidden_units': 256,
 'bert_mask_prob': 0.15,
 'bert_max_len': 100,
 'bert_num_blocks': 2,
 'bert_num_heads': 4,
 'best_metric': 'NDCG@10',
 'decay_step': 25,
 'device': 'cuda',
 'device_idx': '0',
 'enable_lr_schedule': True,
 'experiment_description': 'BT4222',
 'experiment_dir': 'model',
 'gamma': 1.0,
 'log_period_as_iter': 10000,
 'lr': 0.001,
 'metric_ks': [1,
               5,
               10,
               20,
               50,
               100],
 'num_epochs': 100,
 'num_gpu': 1,
 'num_items': 2069,
 'optimizer': 'Adam',
 'random_seed': 10,
 'save_folder': '../data/bert_4_rec_data',
 'test_negative_sample_size': 100,
 'test_negative_sampler_code': 'random',
 'test_negative_sampling_seed': 98765,
 'test_size': 0.19999999999999996,
 'train_negative_sample_size': 0,
 'train_negative_sampler_code': 'random',
 'train_negative_sampling_seed': 0,
 '

In [36]:
trainer.train()

Epoch 11, loss 6.843 : 100%|██████████| 2/2 [00:00<00:00,  4.64it/s]
Val: N@1 0.024, N@5 0.091, N@10 0.109, R@1 0.024, R@5 0.146, R@10 0.203: 100%|██████████| 2/2 [00:00<00:00, 10.08it/s]
Epoch 12, loss 6.735 : 100%|██████████| 2/2 [00:00<00:00,  4.65it/s]
Val: N@1 0.023, N@5 0.085, N@10 0.115, R@1 0.023, R@5 0.142, R@10 0.231: 100%|██████████| 2/2 [00:00<00:00, 10.28it/s]
Epoch 13, loss 6.721 : 100%|██████████| 2/2 [00:00<00:00,  4.53it/s]
Val: N@1 0.044, N@5 0.104, N@10 0.129, R@1 0.044, R@5 0.162, R@10 0.239: 100%|██████████| 2/2 [00:00<00:00,  9.93it/s]
Epoch 14, loss 6.592 : 100%|██████████| 2/2 [00:00<00:00,  4.67it/s]
Val: N@1 0.093, N@5 0.130, N@10 0.156, R@1 0.093, R@5 0.162, R@10 0.239: 100%|██████████| 2/2 [00:00<00:00,  9.96it/s]
Epoch 15, loss 6.587 : 100%|██████████| 2/2 [00:00<00:00,  4.65it/s]
Val: N@1 0.089, N@5 0.126, N@10 0.150, R@1 0.089, R@5 0.158, R@10 0.231: 100%|██████████| 2/2 [00:00<00:00, 10.02it/s]
Epoch 16, loss 6.537 : 100%|██████████| 2/2 [00:00<00:00,  4

In [38]:
trainer.test()

Test best model with test set!
Val: N@1 0.157, N@5 0.217, N@10 0.251, R@1 0.157, R@5 0.270, R@10 0.375: 100%|██████████| 2/2 [00:00<00:00,  9.51it/s]{'Recall@100': 1.0, 'NDCG@100': 0.36983881890773773, 'Recall@50': 0.6609375178813934, 'NDCG@50': 0.3150499314069748, 'Recall@20': 0.5127604305744171, 'NDCG@20': 0.28583210706710815, 'Recall@10': 0.3752604275941849, 'NDCG@10': 0.25090668350458145, 'Recall@5': 0.26953125, 'NDCG@5': 0.21746379882097244, 'Recall@1': 0.15651042014360428, 'NDCG@1': 0.15651042014360428}

